
![ImageName](https://cdn.kesci.com/upload/rudh9dbgjm.jpg)  

**作者：[lqy](https://www.heywhale.com/home/user/profile/5f27fd9633e1be002cc65a1d)** 华东师范大学气象学研究生、和鲸社区气象数据科学频道版主

🐋：本项目来自和鲸社区《[气象训练营⑦：WRF模式后处理](https://www.heywhale.com/home/competition/64478fec113e81a18dc70cd1)》活动，所有教案代码都可以一键跑通，你可以 fork 后在线在线运行、调试学习、完成作业练习。  

学习过程中如果你遇到任何问题，欢迎使用搜索引擎，或在 [讨论区](https://www.heywhale.com/home/competition/forumlist/64478fec113e81a18dc70cd1) 中发帖提出，我们很乐意为你提供帮助。

# 关卡二：绘制 WRF 模拟的降雨量      


上一关我们学习了 WRF 模拟数据的解析，能够提取模拟变量。那么这一关中，我们主要是**实现降雨量模拟结果的计算和可视化**，看看 WRF 模拟的台风降水结果如何？  

在数据处理中，我们特别需要注意变量所表示的**物理含义以及单位**，保证计算结果的正确性。在数据可视化过程中，我们需要注意**投影设置、地图要素添加以及颜色条选取**，让地图信息**更丰富、更直观**。

## 一、两个关键指标：降雨量与降雨强度  

在 WRF 模拟的众多变量中，降雨量和降雨强度是我们非常关心的变量之一，是体现一次降雨过程的重要指标，也是评价模式模拟结果的关键因素。  

我们先来看一下它们的有什么区别：  

* 降雨量：一段时间范围的降雨累积值（而不是某个瞬间），单位通常是 mm。  

* 降雨强度：一段时间范围的降雨累积值除以时长，单位通常是 mm/h。  

由此可见，只要能算出降雨量，就不难得出降雨强度。因此我们这部分的学习重点放在降雨量的计算上。  

**请特别注意**：降雨量是表示一段时间段内的降雨，不同于气压、气温、风速等变量（表示某一时刻的状态）。 因此，降雨量的计算需要考虑起始时刻和结束时刻，这也是特别容易出错的地方。

## 二、两种方法计算 WRF 模拟的降雨量  

在第一关中，我们使用了**两种不同方法（salem和netcdf）** 来提取 WRF 数据变量。尽管有不同的数据读取方式，但读取的数值理论上是一致的。  

本关基于不同数据读取模块，比较其计算 WRF 模拟的降雨量差异，用以验证降雨量计算结果的一致性。  


需要注意的是，WRF 模拟的降雨量是由**三部分（深对流降水+浅对流降水+非对流降水）**组成的，这是大家往往忽略的地方。

### 总降水计算公式  

**总降水 = RAINNC + RAINC + RAINSH**。  

其中：  
- **RAINC（深对流降水）：**ACCUMULATED TOTAL CUMULUS PRECIPITATION。积云深对流过程产生的累积降水量，也就是模式中的积云对流参数化方案导致的降雨 (cu_physics)。对于高分辨率的模拟，比如 dx<5km，通常会将积云对流参数化方案关闭，此时 RAINC 为 0。      
      
- **RAINSH（浅对流降水）：**ACCUMULATED SHALLOW CUMULUS PRECIPITATION。积云对流参数化方案主要是反映深对流的降水过程，但是一些积云对流参数化方案，能够支持浅对流导致的降水，此时总降水还需要加上 RAINSH。WRF 中支持浅对流的参数化方案 (cu_physics) 有以下几种：KF，SAS，G3，BMJ，Tiedtke。WRF 中也有独立于深对流过程的浅对流方案，通过 namelist 中设置 shcu_physics。一般情况下，浅对流产生的降水量较小。根据物理参数化方案的设置可能为 0。         
      
- **RAINNC（非对流降水）：**ACCUMULATED TOTAL GRID SCALE PRECIPITATION。此类降雨来源于云微物理参数化方案 (mp_physics)，如大尺度抬升过程产生的凝结等微物理过程降水，也就是非对流产生的降水。     


在大气中，**降雨可以分为对流性降雨和非对流性降雨，也可以分为固态降水[^1]和液态降水。**    

[^1]: 固态降水例如雪 (SNOWC/SNOWNC)、霰 (GRAUPELC/GRAUPELNC) 等降水，它们是降水的不同相态，已经都包含在 RAINC/RAINNC 中，不需要额外添加。  

在了解WRF模拟的降雨量计算公式后，让我们来看看具体怎么实现。  

### 方法一：基于 salem 读取 & 计算

首先，尝试用 salem 模块读取 WRF 模拟数据，查看总降水量的三个组成部分的数值，并计算总降水量。

In [1]:
# WRF数据目录
wrfout_path = '/home/mw/input/typhoon9537/'

第一次使用 `salem` 需要在 `home` 目录中创建 `.salem_cache` 文件夹，并下载 `salem-sample-data` 的数据包。为了更友好地直接导入 `salem` ，本项目已挂载社区的共享数据集[salem_cache文件集合](https://www.heywhale.com/mw/dataset/64716032b1251d62bfb38aa8/file)，再把该数据集的数据复制到 `/home/mw/.salem_cache/`  内，实现手动下载，避免卡住。

In [2]:
import os
import os.path

if(not os.path.exists('./salem_cache')):
    os.mkdir('./salem_cache') # 创建后它隐形的

! cp -r /home/mw/input/salem_cache05279422/salem_cache/ /home/mw/.salem_cache/ # 拷贝文件
! ls -ll /home/mw/.salem_cache/  # 检查有无成功

total 52924
drwxr-xr-x 3 mw users     4096 May 29 03:42 cache
drwxr-xr-x 2 mw users     4096 May 29 03:42 downloads
drwxr-xr-x 6 mw users     4096 May 29 03:42 salem-sample-data-758f7ddd0fa6b5b1bd4c63b6dcfe8d5eec0f4c59
-rw-r--r-- 1 mw users 54177895 May 29 03:42 salem-sample-data-758f7ddd0fa6b5b1bd4c63b6dcfe8d5eec0f4c59.zip


In [3]:
# 导入模块
import salem
# 用salem包读取数据
ds = salem.open_wrf_dataset(wrfout_path + 'wrfout_d01_2019-08-09_06_00_00')
ds

<xarray.Dataset>
Dimensions:                (south_north: 437, west_east: 447, time: 1, bottom_top: 34, soil_layers: 3, seed_dim: 1)
Coordinates:
    lat                    (south_north, west_east) float32 20.89 ... 32.78
    lon                    (south_north, west_east) float32 116.5 ... 130.3
    xtime                  (time) datetime64[ns] ...
  * time                   (time) datetime64[ns] 2019-08-09T06:00:00
  * west_east              (west_east) float64 -6.69e+05 -6.66e+05 ... 6.69e+05
  * south_north            (south_north) float64 -6.54e+05 ... 6.54e+05
Dimensions without coordinates: bottom_top, soil_layers, seed_dim
Data variables: (12/166)
    LU_INDEX               (time, south_north, west_east) float32 ...
    ZNU                    (time, bottom_top) float32 ...
    ZNW                    (time, bottom_top) float32 ...
    ZS                     (time, soil_layers) float32 ...
    DZS                    (time, soil_layers) float32 ...
    VAR_SSO                (time, south_north, west_east) float32 ...
    ...                     ...
    TK                     (time, bottom_top, south_north, west_east) float32 ...
    WS                     (time, bottom_top, south_north, west_east) float32 ...
    PRESSURE               (time, bottom_top, south_north, west_east) float32 ...
    GEOPOTENTIAL           (time, bottom_top, south_north, west_east) float32 ...
    Z                      (time, bottom_top, south_north, west_east) float32 ...
    SLP                    (time, south_north, west_east) float32 ...
Attributes: (12/135)
    TITLE:                            OUTPUT FROM WRF V4.4.2 MODEL
    START_DATE:                      2019-08-08_18:00:00
    SIMULATION_START_DATE:           2019-08-08_18:00:00
    WEST-EAST_GRID_DIMENSION:        448
    SOUTH-NORTH_GRID_DIMENSION:      438
    BOTTOM-TOP_GRID_DIMENSION:       35
    ...                              ...
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      2
    ETAC:                            0.2
    pyproj_srs:                      +proj=lcc +lat_0=27 +lon_0=123 +lat_1=20...

In [4]:
# 积云深对流过程产生的累积降水量（本案例的模拟积云对流参数化关闭，此处为0）
ds.RAINC.plot()

<Figure size 432x288 with 2 Axes>

In [5]:
# 积云浅对流过程产生的累积降水量（特定的积云对流参数化方案支持浅对流，本案例的模拟中积云对流参数化关闭，此处为0）
ds.RAINSH.plot()

<Figure size 432x288 with 2 Axes>

In [6]:
# 来自云微物理参数化方案的降水
# 由于前两项为0，此处的结果即为总降雨量
ds.RAINNC.plot()

<Figure size 432x288 with 2 Axes>

In [7]:
# 计算从模式初始时刻到当前时刻的累积降雨量
total_rain = ds.RAINC + ds.RAINSH + ds.RAINNC
total_rain

<xarray.DataArray (time: 1, south_north: 437, west_east: 447)>
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)
Coordinates:
    lat          (south_north, west_east) float32 20.89 20.89 ... 32.78 32.78
    lon          (south_north, west_east) float32 116.5 116.6 ... 130.2 130.3
    xtime        (time) datetime64[ns] 2019-08-09T06:00:00
  * time         (time) datetime64[ns] 2019-08-09T06:00:00
  * west_east    (west_east) float64 -6.69e+05 -6.66e+05 ... 6.66e+05 6.69e+05
  * south_north  (south_north) float64 -6.54e+05 -6.51e+05 ... 6.51e+05 6.54e+05

### 方法二：基于 netCDF4 读取 & 计算

我们再来用 netCDF4 模块读取 WRF 模拟数据，结合 wrf-python 提取 WRF 模拟变量，同样查看总降水量的三个组成部分的数值，并计算总降水量。  

一方面是看看结果是否有差异，另一方面是给大家提供两种可参考的处理流程。

In [8]:
# 导入模块
from netCDF4 import Dataset
from wrf import getvar
# 读取 WRF 模拟数据
wrf_file = Dataset(wrfout_path + 'wrfout_d01_2019-08-09_06_00_00')

# 提取降雨量
RAINC = getvar(wrf_file, 'RAINC')
RAINNC = getvar(wrf_file, 'RAINNC')
RAINSH = getvar(wrf_file, 'RAINSH')

# 计算累计降雨量
total_rain = RAINC + RAINSH + RAINNC
total_rain

<xarray.DataArray (south_north: 437, west_east: 447)>
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 116.5 116.6 116.6 ... 130.2 130.3
    XLAT     (south_north, west_east) float32 20.89 20.89 20.89 ... 32.78 32.78
    XTIME    float32 720.0
    Time     datetime64[ns] 2019-08-09T06:00:00
Dimensions without coordinates: south_north, west_east

In [9]:
# 绘制总降雨量的分布
total_rain.plot()

<Figure size 432x288 with 2 Axes>

虽然用了不同的模块去读取 WRF 降雨数据，但结果是完全一样的！ **不同的读取方式并不会导致处理结果差异**，后续大家可以按自己喜欢的工具来处理。  
      
需要注意的是，这个 **预览图的横坐标和纵坐标不是经纬度**，无法直接看出是在哪个地方，因此，还是需要把 WRF 数据投影在地图上进行可视化。

### ✍ 小练习：看看模式初始时刻输出的降雨量      
      
你认为 WRF 模拟起始时刻 `wrfout_d01_2019-08-08_18_00_00` 数据中是否会有降雨？

In [10]:
# ...你的代码...



## 三、获取绘图所需的基础信息

正式绘图前先获取 **经纬度、地图投影和时间** 等信息。  

我们需要注意两点，地图投影通常是选取 WRF 模式运行时设置的投影，WRF 模式的时间是世界协调时（UTC），需要+8小时转换为北京时间。

In [11]:
# 导入模块
from wrf import to_np, getvar, get_cartopy, geo_bounds, cartopy_xlim, cartopy_ylim, latlon_coords

### 1. 提取 WRF 经纬度数组

In [12]:
# 提取WRF模拟的经纬度数组
# 这里通过RAINC数组获取经纬度数组，当然也可以换成其他变量
lats, lons = latlon_coords(RAINC)
print(lats.shape)
print(lons.shape)

(437, 447)
(437, 447)


### 2. 提取 WRF 地理边界

In [13]:
# 提取WRF模拟的地理边界
# 这里通过RAINC数组获取地理边界，当然也可以换成其他变量
bounds = geo_bounds(RAINC)
bounds

GeoBounds(CoordPair(lat=20.89178466796875, lon=116.53927612304688), CoordPair(lat=32.77607727050781, lon=130.26470947265625))

### 3. 提取 WRF 投影设置

In [14]:
# 提取WRF模拟的投影设置
# 在读取的变量属性中，我们可以查看到如下描述投影的字段
# +proj=lcc +lat_0=27 +lon_0=123 +lat_1=20 +lat_2=40 +x_0=0 +y_0=0 +R=6370000 +units=m +no_defs
# get_cartopy可以帮助我们直接提取投影属性，供Cartopy绘图使用
# 这里通过RAINC数组获取地图投影，当然也可以换成其他变量
wrf_proj = get_cartopy(RAINC)
wrf_proj

/opt/conda/lib/python3.7/site-packages/cartopy/io/__init__.py:260: DownloadWarning: Downloading: https://naciscdn.org/naturalearth/110m/physical/ne_110m_coastline.zip
  warnings.warn('Downloading: {}'.format(url), DownloadWarning)


### 4. 提取 WRF 的起止时间

In [15]:
# 提取WRF模拟的起止时间
from datetime import datetime
import numpy as np

# 我们可以用datetime模块输入WRF起始时刻的时间戳
time_start = datetime(2019, 8, 8, 18, 0)

# 将datetime64[ns]格式转换为datetime格式时间戳
time_end = np.array(ds.time)
time_end = datetime.strptime(str(time_end[0].astype('datetime64[ms]')), '%Y-%m-%dT%H:%M:%S.%f')
time_end

datetime.datetime(2019, 8, 9, 6, 0)

通过以上4步，我们已经获取了降雨量绘图的基本信息，后续我们就可以进行地图可视化的编程啦~

### ✍ 小练习：时间戳处理

我们很容易能根据WRF模拟数据的文件名获取当前时刻，但如何智能地获取初始时刻（start_time） 变量？    
      
提示：尽管xtime显示的是当前的时刻，但在description中有初始时刻（start_time）的记录  
![ImageName](https://cdn.kesci.com/upload/image/rqxjnrj30v.png)      


In [16]:
# ...你的代码...


## 四、基于 Cartopy 绘制 WRF 模拟的降雨量

**[Cartopy](https://scitools.org.uk/cartopy/docs/latest/)** 旨在使用最简单直观的方式生成地图，并提供对 matplotlib 友好的协作接口，是 Python 中进行空间数据可视化的常用模块。  


在 Cartopy 中，每种投影都是一个类，被存放在 cartopy.crs 模块中，crs 即坐标参考系统（Coordinate Reference Systems）。 **[支持的投影列表点击这里查看](https://scitools.org.uk/cartopy/docs/v0.15/crs/projections.html)**  


**Cartopy 地图绘制的基本流程**：  

+ 创建画布。  
+ 通过指定 projection 参数，创建 GeoAxes 对象。  
+ 调用 GeoAxes 的方法画图。  
	+ GeoAxes 用法扩展（部分常用）  
	+ set_global：让地图的显示范围扩展至投影的最大范围。例如，对 PlateCarree 投影的 ax 使用后，地图会变成全球的。  
	+ set_extent：给出元组 (x0, x1, y0, y1) 以限制地图的显示范围。  
	+ set_xticks：设置 x 轴的刻度。  
	+ set_yticks：设置 y 轴的刻度。  
	+ gridlines：给地图添加网格线。  
	+ coastlines：在地图上绘制海岸线。  
	+ stock_img：给地图添加低分辨率的地形图背景。  
	+ add_feature：给地图添加特征（例如陆地或海洋的填充、河流等）。  

承接前面第三部分的工作成果，接下来重点就是利用Cartopy模块进行 **WRF 模拟的降雨量空间分布可视化**。

### 1. 导入绘图模块  

我们在这里一次性导入本关所需要使用的模块

In [17]:
# 导入数据读取模块
import numpy as np
import pandas as pd
from netCDF4 import Dataset
import xarray as xr

# 导入可视化模块
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import shapely.geometry as sgeom
import cmaps

# 导入辅助模块
from glob import glob
from copy import copy

### 2. 基于 pcolormesh 绘制格点数据  

WRF 模拟的物理量都属于格点数据，具有空间信息，降雨量亦是如此。首先，我们试试用[**pcolormesh**](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.pcolormesh.html)方法，就是通过格点来展示降雨量数据。

In [18]:
fig = plt.figure(figsize=(10,8))
# 设置地图投影
ax = plt.axes(projection=wrf_proj)
# 设置地图范围
ax.set_xlim(cartopy_xlim(RAINC))
ax.set_ylim(cartopy_ylim(RAINC))
# 绘制降雨量分布（pcolormesh方法进行格点绘制）
im = ax.pcolormesh(to_np(lons), 
                   to_np(lats), 
                   to_np(total_rain), 
                   vmin=0, 
                   vmax=200, 
                   cmap=cmaps.WhiteBlueGreenYellowRed, 
                   transform=ccrs.PlateCarree())
# 为降雨量添加colorbar
cbar = plt.colorbar(im, ax=ax, extend='max', shrink=1)
cbar.set_label('Rainfall (mm)', fontdict={'size':20})
cbar.ax.tick_params(labelsize=20)
# 添加经纬度网格线
ax.gridlines(color='black', linestyle='dotted')
# 设置标签大小
plt.tick_params(labelsize=15)
# 添加标题
plt.title('08-08 1800 - 08-09 0600(UTC)', loc='left', fontsize=20)
plt.show()

/opt/conda/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1598: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  shading=shading)


<Figure size 720x576 with 2 Axes>

### ✍ 小练习：基于 contourf （等值线填充）绘制格点数据  
      
除了用[**pcolormesh**](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.pcolormesh.html)方法以外，还可以用[**contourf**](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.contourf.html#matplotlib.pyplot.contourf)，即通过等值线填充来绘制，两者的示意图如下图所示。  

![Image Name](https://cdn.kesci.com/upload/runjbw8o27.png)  


In [19]:
### （你的代码） ###
## 通过等值线填充绘制降雨量分布




### 优化：添加地理矢量数据  

没有地图要素数据，我们很难直观看出这个台风降雨具体在哪些区域。因此，**提供地图要素（国界线、海岸线等）作为背景信息是十分有必要的**。

In [20]:
# 创建一个画布
fig = plt.figure(figsize=(10,8))
# 设置地图投影
ax = plt.axes(projection=wrf_proj)
# 设置地图范围
ax.set_xlim(cartopy_xlim(RAINC))
ax.set_ylim(cartopy_ylim(RAINC))

# 读取国界线
province = shpreader.Reader('/home/mw/input/data5246/中国地图/China_provinces/China_provinces.shp')
# 读取九段线
nineline = shpreader.Reader('/home/mw/input/data5246/中国地图/China_10-dash_line/China_10-dash_line.shp')
# 绘制国界线
ax.add_geometries(province.geometries(), 
                  crs=ccrs.PlateCarree(), 
                  linewidth=0.5, 
                  edgecolor='k',
                  facecolor='none',
                  zorder=2)
# 绘制九段线
ax.add_geometries(nineline.geometries(), 
                  crs=ccrs.PlateCarree(), 
                  linewidth=0.5,
                  color='k',
                  zorder=2)
# 绘制降雨量分布（pcolormesh方法进行格点绘制）
im = ax.pcolormesh(to_np(lons), 
                   to_np(lats), 
                   to_np(total_rain), 
                   vmin=0, 
                   vmax=200, 
                   cmap=cmaps.WhiteBlueGreenYellowRed, 
                   transform=ccrs.PlateCarree())
# 为降雨量添加colorbar
cbar = plt.colorbar(im, ax=ax, extend='max', shrink=1)
cbar.set_label('Rainfall (mm)', fontdict={'size':20})
cbar.ax.tick_params(labelsize=20)
# 添加经纬度网格线
ax.gridlines(color='black', linestyle='dotted')
# 设置标签大小
plt.tick_params(labelsize=15)
# 添加标题
plt.title('08-08 1800 - 08-09 0600(UTC)', loc='left', fontsize=20)
plt.show()

/opt/conda/lib/python3.7/site-packages/cmaps/cmaps.py:2541: UserWarning: Trying to register the cmap 'WhiteBlueGreenYellowRed' which already exists.
  matplotlib.cm.register_cmap(name=cname, cmap=cmap)


<Figure size 720x576 with 2 Axes>

### 优化：对非等经纬度投影添加经纬度标签  

由于Cartopy尚不能对非等经纬投影添加经纬度标签，因此我们需要自定义函数来额外实现这一功能。

In [21]:
'''
参考Andrew Dawson 提供的解决方法: https://gist.github.com/ajdawson/dd536f786741e987ae4e
'''
# 给出一个假定为矩形的LineString，返回对应于矩形给定边的直线。
def find_side(ls, side):
    """
    Given a shapely LineString which is assumed to be rectangular, return the
    line corresponding to a given side of the rectangle.
    """
    minx, miny, maxx, maxy = ls.bounds
    points = {'left': [(minx, miny), (minx, maxy)],
              'right': [(maxx, miny), (maxx, maxy)],
              'bottom': [(minx, miny), (maxx, miny)],
              'top': [(minx, maxy), (maxx, maxy)],}
    return sgeom.LineString(points[side])

# 在兰伯特投影的底部X轴上绘制刻度线
def lambert_xticks(ax, ticks):
    """Draw ticks on the bottom x-axis of a Lambert Conformal projection."""
    te = lambda xy: xy[0]
    lc = lambda t, n, b: np.vstack((np.zeros(n) + t, np.linspace(b[2], b[3], n))).T
    xticks, xticklabels = _lambert_ticks(ax, ticks, 'bottom', lc, te)
    ax.xaxis.tick_bottom()
    ax.set_xticks(xticks)
    ax.set_xticklabels([ax.xaxis.get_major_formatter()(xtick) for xtick in xticklabels])

# 在兰伯特投影的左侧y轴上绘制刻度线
def lambert_yticks(ax, ticks):
    """Draw ricks on the left y-axis of a Lamber Conformal projection."""
    te = lambda xy: xy[1]
    lc = lambda t, n, b: np.vstack((np.linspace(b[0], b[1], n), np.zeros(n) + t)).T
    yticks, yticklabels = _lambert_ticks(ax, ticks, 'left', lc, te)
    ax.yaxis.tick_left()
    ax.set_yticks(yticks)
    ax.set_yticklabels([ax.yaxis.get_major_formatter()(ytick) for ytick in yticklabels])

# 获取兰伯特投影中底部X轴或左侧y轴的刻度线位置和标签
def _lambert_ticks(ax, ticks, tick_location, line_constructor, tick_extractor):
    """Get the tick locations and labels for an axis of a Lambert Conformal projection."""
    outline_patch = sgeom.LineString(ax.outline_patch.get_path().vertices.tolist())
    axis = find_side(outline_patch, tick_location)
    n_steps = 30
    extent = ax.get_extent(ccrs.PlateCarree())
    _ticks = []
    for t in ticks:
        xy = line_constructor(t, n_steps, extent)
        proj_xyz = ax.projection.transform_points(ccrs.Geodetic(), xy[:, 0], xy[:, 1])
        xyt = proj_xyz[..., :2]
        ls = sgeom.LineString(xyt.tolist())
        locs = axis.intersection(ls)
        if not locs:
            tick = [None]
        else:
            tick = tick_extractor(locs.xy)
        _ticks.append(tick[0])
    # Remove ticks that aren't visible: 
    ticklabels = copy(ticks)
    while True:
        try:
            index = _ticks.index(None)
        except ValueError:
            break
        _ticks.pop(index)
        ticklabels.pop(index)
    return _ticks, ticklabels

In [22]:
# 创建一个画布
fig = plt.figure(figsize=(10,8))
# 设置地图投影
ax = plt.axes(projection=wrf_proj)
# 设置地图范围
ax.set_xlim(cartopy_xlim(RAINC))
ax.set_ylim(cartopy_ylim(RAINC))

# 读取国界线
province = shpreader.Reader('/home/mw/input/data5246/中国地图/China_provinces/China_provinces.shp')
# 读取九段线
nineline = shpreader.Reader('/home/mw/input/data5246/中国地图/China_10-dash_line/China_10-dash_line.shp')
# 绘制国界线
ax.add_geometries(province.geometries(), 
                  crs=ccrs.PlateCarree(), 
                  linewidth=0.5, 
                  edgecolor='k',
                  facecolor='none',
                  zorder=2)
# 绘制九段线
ax.add_geometries(nineline.geometries(), 
                  crs=ccrs.PlateCarree(), 
                  linewidth=0.5,
                  color='k',
                  zorder=2)
# 绘制降雨量分布（pcolormesh方法进行格点绘制）
im = ax.pcolormesh(to_np(lons), 
                   to_np(lats), 
                   to_np(total_rain), 
                   vmin=0, 
                   vmax=200, 
                   cmap=cmaps.WhiteBlueGreenYellowRed, 
                   transform=ccrs.PlateCarree())
# 为降雨量添加colorbar
cbar = plt.colorbar(im, ax=ax, extend='max', shrink=1)
cbar.set_label('Rainfall (mm)', fontdict={'size':20})
cbar.ax.tick_params(labelsize=20)
# 添加经纬度网格线
ax.gridlines(color='black', linestyle='dotted')
# 添加经纬度标签
xticks = list(np.arange(100,140,2))
yticks = list(np.arange(20,40,2))
fig.canvas.draw()
ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
lambert_xticks(ax, xticks)
lambert_yticks(ax, yticks)
# 设置标签大小
plt.tick_params(labelsize=15)
# 添加标题
plt.title('08-08 1800 - 08-09 0600(UTC)', loc='left', fontsize=20)
plt.show()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: The outline_patch property is deprecated. Use GeoAxes.spines['geo'] or the default Axes properties instead.


<Figure size 720x576 with 2 Axes>

### 小练习：绘制小时降雨量与累积降雨量（用组图形式展示）  

时间范围：2019年8月8日19时（UTC）-2019年8月9日6时（UTC），区域范围与上图保持一致。

In [23]:
### （你的代码） ###
## 绘制累积降雨量组图


## 绘制小时降雨量组图



恭喜你完成了 WRF 后处理训练营第二关的学习材料，了解 **WRF 模拟的降雨量的组成与计算**，并且学会了在**地图上可视化 WRF 模拟数据**的技能。

## 闯关题  

### STEP1：根据要求完成题目

1. 案例中绘图用的是投影是？      
A. 兰伯特方位等积投影      
B. 兰勃特等角圆锥投影      
C. 墨卡托投影（正轴等角切圆柱投影）      
D. 等距方位投影      
      
2. 累积降雨量的正确计算公式为？      
A. 总降水 = RAINNC + RAINC      
B. 总降水 = RAINNC + RAINSH      
C. 总降水 = RAINC + RAINSH      
D. 总降水 = RAINNC + RAINC + RAINSH      
          
3. 从 WRF 模拟初始时刻 2019-08-08_18_00_00（UTC）到 2019-08-09_06_00_00（UTC），累积降雨量最大值和小时降雨量最大值分别为多少 mm？（使用 round 取整）  

提示： 可以用 [xarray.DataArray.diff](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.diff.html) 来求差值（根据累积降雨量计算小时降雨量） 

In [ ]:
# 填入你的答案，注意大小写
answer_1 = ''
answer_2 = ''
answer_3 = ''  # 累积降雨量最大值（使用 round 取整）
answer_4 = ''  # 小时降雨量最大值（使用 round 取整）

### STEP2：将结果保存为 csv 文件  

直接运行下方这个代码 cell，别改它。

In [ ]:
# 生成 csv 作业答案文件
def save_csv(a1, a2, a3, a4):
    import pandas as pd
    df = pd.DataFrame({"id": ["q1", "q2", "q3", "q4"], "answer": [a1, a2, a3, a4]})
    df.to_csv("answer_wrf_2.csv", index=None)

# 生成答案文件
# 该文件在左侧文件树project工作区下，你可以自行右击下载或者读取查看
save_csv(answer_1, answer_2, answer_3, answer_4)  

### STEP3: 提交 csv 文件，获取分数结果  


你的 csv 答案文件已经准备完毕了，最后让我们提交答案文件，看看是否正确。  

提交方法：  

**1、拷贝提交 token**  

去对应关卡的 [提交页面](https://www.heywhale.com/home/competition/64478fec113e81a18dc70cd1/submit)，找到对应关卡，看到了你的 token 嘛？  

拷贝它。  

记得：每个关卡的 token 不一样。  

**2、下方 cell 里，拿你拷贝的 token 替换掉 XXXXXXX， 然后 Ctrl+Enter 运行 。**

In [ ]:
# 运行这个Cell 下载提交工具

!wget -nv -O heywhale_submit https://cdn.kesci.com/submit_tool/v4/heywhale_submit&&chmod +x heywhale_submit

# 运行提交工具
# 把下方XXXXXXX替换为你的 Token
# 改完看起来像是：!./heywhale_submit -token 586eeef71cb92941 -file answer_wrf_2.csv

!./heywhale_submit -token XXXXXXX -file answer_wrf_2.csv  # 替换XXXXXXX；注意不可增减任何空格或其他字符

运行成功、显示提交完成后，即可去 [提交页面](https://www.heywhale.com/home/competition/64478fec113e81a18dc70cd1/submit) 看成绩。满分即可进入下一关。  

没有成功也不怕，看下报错信息，对照《💡 [【提交出错】常见提交错误与排查建议》](https://www.heywhale.com/home/competition/forum/645a264a5ce8710c5bafb933) 帮助帖排查下，改完重新提交咯。

## ⚡ 下一关预告：  

下一关我们将介绍 WRF 模拟的风场绘制，依然会涉及到本关中的地图绘制要点，看看 WRF 对台风风场的模拟能力如何？